In [1]:
import numpy as np
import sympy as sym
import json
import sympy as sym
import pydae.buildfast as db
from pydae.grid_bpu import bpu

In [2]:
file_path = 'newengland.json'
bpu_obj = bpu(data_input=file_path)

In [3]:
g_list = bpu_obj.dae['g'] 
h_dict = bpu_obj.dae['h_dict']
f_list = bpu_obj.dae['f']
x_list = bpu_obj.dae['x']
params_dict = bpu_obj.dae['params_dict']

sys = {'name':'newengland',
       'params_dict':params_dict,
       'f_list':f_list,
       'g_list':g_list,
       'x_list':x_list,
       'y_ini_list':bpu_obj.dae['y_ini'],
       'y_run_list':bpu_obj.dae['y_run'],
       'u_run_dict':bpu_obj.dae['u_run_dict'],
       'u_ini_dict':bpu_obj.dae['u_ini_dict'],
       'h_dict':h_dict}

sys = db.system(sys)
#db.sys2num(sys)

In [13]:
from sympy import SparseMatrix, Matrix

In [16]:
Gy_ini_sp = SparseMatrix(sys['Gy_ini'])

In [25]:
from sympy.matrices.sparsetools import _doktocsr

Gy_ini_csr = _doktocsr(Gy_ini_sp)

In [33]:
Gy_ini_csr[3]

[170, 170]

In [42]:
def getIndex(s, i): 
  
    # If input is invalid. 
    if s[i] != '(':
        return -1
  
    # Create a deque to use it as a stack. 
    d = deque() 
  
    # Traverse through all elements 
    # starting from i. 
    for k in range(i, len(s)): 
  
        # Pop a starting bracket 
        # for every closing bracket 
        if s[k] == ')': 
            d.popleft() 
  
        # Push all starting brackets 
        elif s[k] == '(': 
            d.append(s[i]) 
  
        # If deque becomes empty 
        if not d: 
            return k 
  
    return -1
  
def arg2np(string,function_name):
    idx_end = 0
    for it in range(3):
        if function_name in string[idx_end:]:
            #print(string[idx_end:])
            idx_ini = string.find(f'{function_name}(',idx_end)+len(f'{function_name}(')
            idx_end = getIndex(string, idx_ini-1)
            string = string.replace(string[idx_ini:idx_end],f'np.array([{string[idx_ini:idx_end]}])')
    else: pass
    return string   

def arg2cp(string,function_name,i_pwise):
    idx_end = 0
    for it in range(3):
        if function_name in string[idx_end:]:
            #print(string[idx_end:])
            idx_ini = string.find(f'{function_name}(',idx_end)+len(f'{function_name}(')
            idx_end = getIndex(string, idx_ini-1)
            string_inline = string.replace(string[idx_ini:idx_end],'pwise[0]').replace(f'{function_name}(','').replace(')','')
            string_pwise = string[idx_ini:idx_end]
    else: pass
    return string_pwise,string_inline 

In [45]:
from collections import deque 

string = str(sys['g'][-3])

ValueError: too many values to unpack (expected 2)

In [47]:
string_pwise,string_inline =  arg2cp(string,'Piecewise',0)
string_pwise_list = string_pwise.split(',')

In [50]:
string_pwise_list

for it in range(int(len(string_pwise_list)/2)):
    string_pwise_list[2*it] = string_pwise_list[2*it][1:] 
    string_pwise_list[2*it+1] = string_pwise_list[2*it+1][:-1] 
    
    
string_pwise_list    

['-V_lim_4',
 ' V_lim_4 < -K_stab_4*(T_1_4*(-x_lead_4 + z_wo_4)/T_2_4 + x_lead_4)',
 '(V_lim_4',
 ' V_lim_4 < K_stab_4*(T_1_4*(-x_lead_4 + z_wo_4)/T_2_4 + x_lead_4)',
 '(K_stab_4*(T_1_4*(-x_lead_4 + z_wo_4)/T_2_4 + x_lead_4)',
 ' True']

In [6]:
from cffi import FFI

src = """

/* Define the C struct */
typedef struct my_struct {
   int    i1;
   float  f2;
   double d3;
   float  af4[7]; // arrays are supported
} my_struct;

/* Define a callback function */
typedef double (*my_func)(my_struct*, size_t);
"""

ffi = FFI()
ffi.cdef(src)

# Get the function signature from *my_func*
sig = cffi_utils.map_type(ffi.typeof('my_func'), use_record_dtype=True)

# Make the cfunc
from numba import cfunc, carray

@cfunc(sig)
def foo(ptr, n):
   base = carray(ptr, n)  # view pointer as an array of my_struct
   tmp = 0
   for i in range(n):
      tmp += base[i].i1 * base[i].f2 / base[i].d3
      tmp += base[i].af4.sum()  # nested arrays are like normal numpy array
   return tmp

NameError: name 'cffi_utils' is not defined

In [7]:
import os

import numba as nb
import numpy as np
from cffi import FFI

from numpy.random import PCG64

ffi = FFI()
if os.path.exists('./distributions.dll'):
    lib = ffi.dlopen('./distributions.dll')
elif os.path.exists('./libdistributions.so'):
    lib = ffi.dlopen('./libdistributions.so')
else:
    raise RuntimeError('Required DLL/so file was not found.')

ffi.cdef("""
double random_standard_normal(void *bitgen_state);
""")
x = PCG64()
xffi = x.cffi
bit_generator = xffi.bit_generator

random_standard_normal = lib.random_standard_normal


def normals(n, bit_generator):
    out = np.empty(n)
    for i in range(n):
        out[i] = random_standard_normal(bit_generator)
    return out


normalsj = nb.jit(normals, nopython=True)

# Numba requires a memory address for void *
# Can also get address from x.ctypes.bit_generator.value
bit_generator_address = int(ffi.cast('uintptr_t', bit_generator))

norm = normalsj(1000, bit_generator_address)
print(norm[:12])

RuntimeError: Required DLL/so file was not found.

In [18]:
import cffi

ffi = cffi.FFI()
defs="void foo_f(double a,double *b);"
ffi.cdef(defs, override=True)
source="""
void foo_f(double a,double *b){
  b[0]+=a;
  }
"""
ffi.set_source(module_name="foo",source=source)
ffi.compile()

VerificationError: LinkError: command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\BuildTools\\VC\\Tools\\MSVC\\14.27.29110\\bin\\HostX86\\x64\\link.exe' failed with exit status 1104

In [27]:
import numpy as np
import numba as nb
from numba.core.typing import cffi_utils as cffi_support
import cffi
ffi = cffi.FFI()
import foo

cffi_support.register_module(foo)
foo_f = foo.lib.foo_f

@nb.njit("float64(float64,float64)")
def method_using_arrays(a,b):
    b_arr=np.empty(1,dtype=np.float64)
    b_arr[0]=b
    b_arr_ptr=b_wrap=ffi.from_buffer(b_arr)
    foo_f(a,b_arr_ptr)
    return b_arr[0]

In [26]:
method_using_arrays(np.array([2.0]),np.array([3.0]))

TypeError: No matching definition for argument type(s) array(float64, 1d, C), array(float64, 1d, C)